In [ ]:
import time
from functools import wraps
import random
import os

def print_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        duration_time = end_time - start_time
        print("execute time running %s: %s seconds" % (func.__name__, duration_time))
        return result
 
    return wrapper

In [ ]:
import requests,json
from multiprocessing import Pool
import pickle as pkl

def get_problem_by_slug(slug):
    session = requests.Session()
    user_agent = r'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'

    url = "https://leetcode.com/graphql"
    params = {'operationName': "getQuestionDetail",
        'variables': {'titleSlug': slug},
        'query': '''query getQuestionDetail($titleSlug: String!) {
            question(titleSlug: $titleSlug) {
                questionFrontendId
                likes
                dislikes
                difficulty
            }
        }'''
    }

    json_data = json.dumps(params).encode('utf8')
                        
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug}
    resp = session.post(url, data = json_data, headers = headers, timeout = 10)
    content = resp.json()

    # 题目详细信息
    question = content['data']['question']
    like_rate=(content['data']['question']['likes']/(content['data']['question']['dislikes']+content['data']['question']['likes']))
    return (content['data']['question']['questionFrontendId'],slug,content['data']['question']['difficulty'],like_rate)
@print_time
def get_problems(url = "https://leetcode.com/api/problems/all/"):
    session = requests.Session()
    user_agent = r'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'

    headers = {'User-Agent': user_agent, 'Connection': 'keep-alive'}
    resp = session.get(url, headers = headers, timeout = 10)
       
    question_list = json.loads(resp.content.decode('utf-8'))

    slugs=[question['stat']['question__title_slug'] for question in question_list['stat_status_pairs']]
    pool=Pool()
    res = pool.map(get_problem_by_slug,slugs)
    pool.close()
    pool.join()
    return res



In [ ]:
liked_filename='leetcode_liked_info.pkl'
if os.path.exists(liked_filename) and False:
    with open(liked_filename,'rb') as f:
        a=pkl.load(f)
else:
    a=get_problems()
    a.sort(reverse=True,key=lambda x:x[3])
    with open(liked_filename,'wb') as f:
        pkl.dump(a, f)

In [ ]:
hard=[line[:-2] for line in a if line[2]=='Hard' and int(line[0])<1000]
# import pandas as pd
# pd.set_option('display.max_rows', 500)
# pd.DataFrame(hard)
for line in hard:
    print(line[0],'\t',line[1])

In [ ]:
a

In [ ]:
len(a)

In [ ]:
with open('tmp.txt', 'w') as f:
    for i, x in enumerate(a):
#         if i > 3:
#             break
#         print(x[0], x[1], x[2])
        f.write(x[0] + ',' + x[1] + ',' + x[2] + ',' + str(x[3]) + '\n')

In [ ]:
print(filtered)

In [ ]:
set([1])[0]